In [1]:
import pandas as pd
import math
import numpy as np
from astropy.coordinates import SkyCoord
from astropy.io import fits
import healpy as hp
import matplotlib.pyplot as plt
import time
import sys
from pixell import enmap, enplot, reproject, utils, curvedsky, wcsutils
from matplotlib import cm
from scipy.optimize import curve_fit
from scipy.linalg import sqrtm
from tqdm import tqdm
import random
import time
import multiprocessing as mp
import numpy as np
import random
import time
import multiprocessing as mp
import numpy as np

# Rad: 5, 10

In [2]:
def import_map(filepath, beam):
    pl = hp.read_map(filepath)
    pl[(pl==-32768) | np.isnan(pl)] = 0
    try:
        shape,wcs = enmap.fullsky_geometry(res=int(beam)*utils.arcmin,proj='car')
        map_pix = reproject.enmap_from_healpix(pl,shape=shape,wcs=wcs)
    except:
        shape,wcs = enmap.fullsky_geometry(res=int(beam+1)*utils.arcmin,proj='car')
        map_pix = reproject.enmap_from_healpix(pl,shape=shape,wcs=wcs)
    return map_pix


def extract_intensity(map_pix, ras, decs, beam):
    N = len(ras)
    stack_CMB_kSZ           = 0
    stack_CMB_kSZ_deproject = 0
    c = 0

    signal = []
    signal_ring = []
    ras_used = []
    decs_used = []
    for i in tqdm(range(N)):
        stamp = reproject.thumbnails(map_pix,coords = np.deg2rad([decs[i], ras[i]]),r=beam*utils.arcmin)
        stamp2 = reproject.thumbnails(map_pix,coords = np.deg2rad([decs[i], ras[i]]),r=beam*2*utils.arcmin)

        if stamp is None:
            signal.append(-9999)
            signal_ring.append(-9999)
        elif stamp[0][0][0]==0.0:
            signal.append(-9999)
            signal_ring.append(-9999)
        else:
            signal_sum = sum(sum(stamp[0]))
            signal2_sum = sum(sum(stamp2[0]))
            num_pix_signal = len(stamp[0][0])*len(stamp[0][0])
            num_pix_signal2 = len(stamp2[0][0])*len(stamp2[0][0])            
            signal.append(signal_sum/num_pix_signal)
            signal_ring.append((signal2_sum-signal_sum)/(num_pix_signal2-num_pix_signal))
        ras_used.append(ras[i])
        decs_used.append(decs[i])

    return signal, signal_ring, ras_used, decs_used


def output_signal(signal,signal_ring,output_fp,output_fp_signal):
    np.savetxt(output_fp,
               signal,
               delimiter=", ",
               fmt='% s')
    np.savetxt(output_fp_signal,
               signal_ring,
               delimiter=", ",
               fmt='% s')

In [4]:
real = pd.read_csv("data/union3.csv", skiprows = 0)
real6 = pd.read_csv("data/sdss.csv", skiprows = 0)

# run catalogue 3 --> 
# do one run that is [::4] ~50,000 objects
# do one run with ALL objects ~200,000 objects

# Zou HSC, z > 1
real1 = real[(real.z > 1) & (real.cat == 3)]

# WH22, z > 1
real2 = real[(real.z > 1) & (real.cat == 4)]

# Zou DESI, 0.6 < z < 1
real3 = real[(real.z < 1) & (real.z > 0.6) & (real.cat == 2)]
real3_c = real[(real.z < 1) & (real.z > 0.6) & (real.cat == 2)][::4]

# Zou DES, 0.6 < z < 1
real4 = real[(real.z < 1) & (real.z > 0.6) & (real.cat == 1)]

# WH22, 0.6 < z < 1
real5 = real[(real.z < 1) & (real.z > 0.6) & (real.cat == 4)]

# SDSS 0.05 < z < 0.6
real6 = real6[(real6.z < 0.6) & (real6.z > 0.05)]

# Zou DESI, 0.05 < z < 0.6
real7 = real[(real.z < 0.6) & (real.z > 0.05) & (real.cat == 2)]

# intersection of DESI, DES, WH22 in same area (look at prob in those objects that are shared vs. those not shared)
# mass cuts in DESI --> instead of random 100k points, take richest 100k ones

In [ ]:
# real = pd.read_csv('data/union2.csv',skiprows=0,index_col=0)[::3]
ras_real = real3['amf_ra'].reset_index(drop=True)##[real.z>0.75][::2].reset_index(drop=True)
decs_real = real3['amf_dec'].reset_index(drop=True)##[real.z>0.75][::2].reset_index(drop=True)

ras_all = ras_real
decs_all = decs_real

maps = "data/IRIS_nohole_1_2048_v2.fits"
beam=10
map_pix = import_map(maps, int(beam))

outputs = "output/510u3C3/ir12.csv"
outputs2 = "output/510u3C3/ir12_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

In [ ]:
# real = pd.read_csv('data/union2.csv',skiprows=0,index_col=0)[::3]
ras_real = real3['amf_ra'].reset_index(drop=True)##[real.z>0.75][::2].reset_index(drop=True)
decs_real = real3['amf_dec'].reset_index(drop=True)##[real.z>0.75][::2].reset_index(drop=True)

ras_all = ras_real
decs_all = decs_real

maps = "data/IRIS_nohole_2_2048_v2.fits"
beam=10
map_pix = import_map(maps, int(beam))

outputs = "output/510u3C3/ir25.csv"
outputs2 = "output/510u3C3/ir25_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

In [ ]:
#real = pd.read_csv('data/union2.csv',skiprows=0,index_col=0)[::3]
ras_real = real3['amf_ra'].reset_index(drop=True)##[real.z>0.75][::2].reset_index(drop=True)
decs_real = real3['amf_dec'].reset_index(drop=True)##[real.z>0.75][::2].reset_index(drop=True)

ras_all = ras_real
decs_all = decs_real

maps = "data/IRIS_nohole_3_2048_v2.fits"
beam=10
map_pix = import_map(maps, int(beam))

outputs = "output/510u3C3/ir60.csv"
outputs2 = "output/510u3C3/ir60_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

In [ ]:
#real = pd.read_csv('data/union2.csv',skiprows=0,index_col=0)[::3]
ras_real = real3['amf_ra'].reset_index(drop=True)##[real.z>0.75][::2].reset_index(drop=True)
decs_real = real3['amf_dec'].reset_index(drop=True)##[real.z>0.75][::2].reset_index(drop=True)

ras_all = ras_real
decs_all = decs_real

maps = "data/IRIS_nohole_4_2048_v2.fits"
beam=10
map_pix = import_map(maps, int(beam))

outputs = "output/510u3C3/ir100.csv"
outputs2 = "output/510u3C3/ir100_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

---

In [ ]:
voids = pd.read_csv('data/voids.csv',skiprows=0,index_col=0)
ras_voids = voids['ras'].reset_index(drop=True)
decs_voids = voids['decs'].reset_index(drop=True)

ras_all = ras_voids
decs_all = decs_voids

maps = "data/IRIS_nohole_1_2048_v2.fits"
beam=10
map_pix = import_map(maps, int(beam))

outputs = "output/510u3/iv12.csv"
outputs2 = "output/510u3/iv12_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

In [ ]:
voids = pd.read_csv('data/voids.csv',skiprows=0,index_col=0)
ras_voids = voids['ras'].reset_index(drop=True)
decs_voids = voids['decs'].reset_index(drop=True)

ras_all = ras_voids
decs_all = decs_voids

maps = "data/IRIS_nohole_2_2048_v2.fits"
beam=10
map_pix = import_map(maps, int(beam))

outputs = "output/510u3/iv25.csv"
outputs2 = "output/510u3/iv25_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

In [ ]:
voids = pd.read_csv('data/voids.csv',skiprows=0,index_col=0)
ras_voids = voids['ras'].reset_index(drop=True)
decs_voids = voids['decs'].reset_index(drop=True)

ras_all = ras_voids
decs_all = decs_voids

maps = "data/IRIS_nohole_3_2048_v2.fits"
beam=10
map_pix = import_map(maps, int(beam))

outputs = "output/510u3/iv60.csv"
outputs2 = "output/510u3/iv60_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

In [ ]:
voids = pd.read_csv('data/voids.csv',skiprows=0,index_col=0) 
ras_voids = voids['ras'].reset_index(drop=True)
decs_voids = voids['decs'].reset_index(drop=True)

ras_all = ras_voids
decs_all = decs_voids

maps = "data/IRIS_nohole_4_2048_v2.fits"
beam=10
map_pix = import_map(maps, int(beam))

outputs = "output/510u3/iv100.csv"
outputs2 = "output/510u3/iv100_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

In [5]:
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
from astropy import units as u
from tqdm import tqdm

# galactic coordinates
glon = []
glat = []
for lon in tqdm(np.linspace(0,180,540)):
    for lat in np.linspace(15,90,224):
        glon.append(lon)
        glat.append(lat)
glon2 = [i*u.degree for i in glon]
glat2 = [i*u.degree for i in glat]

# convert to equitorial coordinates
c = SkyCoord(glon2, glat2, frame='galactic')
ras = c.icrs.ra.deg
decs = c.icrs.dec.deg

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 540/540 [00:00<00:00, 8401.61it/s]


In [3]:
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
from astropy import units as u

jiggles = pd.read_csv('output/Locations/C3_wA/Allsky/jiggles_80.csv', skiprows = 0)
# jiggles[jiggles.glon>360] = 360
# jiggles[jiggles.glat>90] = 90

glon = jiggles.glon.values
glat = jiggles.glat.values

glon2 = [i*u.degree for i in glon]
glat2 = [i*u.degree for i in glat]

c = SkyCoord(glon2, glat2, frame='galactic')
ras = c.icrs.ra.deg
decs = c.icrs.dec.deg

In [4]:
# ras_fsky = pd.read_csv('data/full_sky_ras.csv').reset_index(drop=True)
# decs_fsky = pd.read_csv('data/full_sky_decs.csv').reset_index(drop=True)

ras_all = np.array(ras)#np.array(ras)#ras_fsky['ras'].values
decs_all = np.array(decs)#decs_fsky['decs'].values

maps = "data/IRIS_nohole_1_2048_v2.fits"
beam=10
map_pix = import_map(maps, int(beam))

outputs = "output/Locations/C3_wA/Allsky/JigglesData/if12.csv"
outputs2 = "output/Locations/C3_wA/Allsky/JigglesData/if12_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

Preparing SHT
T -> alm
float64 complex128
Computing pixel positions
Computing rotated positions
Projecting


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 869311/869311 [3:37:41<00:00, 66.55it/s]


In [5]:
# ras_fsky = pd.read_csv('data/full_sky_ras.csv').reset_index(drop=True)
# decs_fsky = pd.read_csv('data/full_sky_decs.csv').reset_index(drop=True)

ras_all = np.array(ras)#np.array(ras)#ras_fsky['ras'].values
decs_all = np.array(decs)#decs_fsky['decs'].values

maps = "data/IRIS_nohole_2_2048_v2.fits"
beam=10
map_pix = import_map(maps, int(beam))

outputs = "output/Locations/C3_wA/Allsky/JigglesData/if25.csv"
outputs2 = "output/Locations/C3_wA/Allsky/JigglesData/if25_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

Preparing SHT
T -> alm
float64 complex128
Computing pixel positions
Computing rotated positions
Projecting


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 869311/869311 [3:30:04<00:00, 68.97it/s]


In [4]:
# ras_fsky = pd.read_csv('data/full_sky_ras.csv').reset_index(drop=True)
# decs_fsky = pd.read_csv('data/full_sky_decs.csv').reset_index(drop=True)

ras_all = np.array(ras)#np.array(ras)#ras_fsky['ras'].values
decs_all = np.array(decs)#decs_fsky['decs'].values

maps = "data/IRIS_nohole_3_2048_v2.fits"
beam=10
map_pix = import_map(maps, int(beam))

outputs = "output/Locations/C3_wA/Allsky/JigglesData/if60.csv"
outputs2 = "output/Locations/C3_wA/Allsky/JigglesData/if60_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

Preparing SHT
T -> alm
float64 complex128
Computing pixel positions
Computing rotated positions
Projecting


100%|█████████████████████████████████████████████████████████████████████████| 869311/869311 [3:41:36<00:00, 65.38it/s]


In [5]:
# ras_fsky = pd.read_csv('data/full_sky_ras.csv').reset_index(drop=True)
# decs_fsky = pd.read_csv('data/full_sky_decs.csv').reset_index(drop=True)

ras_all = np.array(ras)#np.array(ras)#ras_fsky['ras'].values
decs_all = np.array(decs)#decs_fsky['decs'].values

maps = "data/IRIS_nohole_4_2048_v2.fits"
beam=10
map_pix = import_map(maps, int(beam))

outputs = "output/Locations/C3_wA/Allsky/JigglesData/if100.csv"
outputs2 = "output/Locations/C3_wA/Allsky/JigglesData/if100_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

Preparing SHT
T -> alm
float64 complex128
Computing pixel positions
Computing rotated positions
Projecting


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 869311/869311 [4:11:45<00:00, 57.55it/s]


## Planck

### Real

In [6]:
def import_map(filepath, beam):
    pl = hp.read_map(filepath)
    pl[pl<-1e23]=0
    try:
        shape,wcs = enmap.fullsky_geometry(res=int(beam)*utils.arcmin,proj='car')
        map_pix = reproject.enmap_from_healpix(pl,shape=shape,wcs=wcs)
    except:
        shape,wcs = enmap.fullsky_geometry(res=int(beam+1)*utils.arcmin,proj='car')
        map_pix = reproject.enmap_from_healpix(pl,shape=shape,wcs=wcs)
    return map_pix


def extract_intensity(map_pix, ras, decs, beam):
    N = len(ras)
    stack_CMB_kSZ           = 0
    stack_CMB_kSZ_deproject = 0
    c = 0

    signal = []
    signal_ring = []
    ras_used = []
    decs_used = []
    for i in tqdm(range(N)):
        stamp = reproject.thumbnails(map_pix,coords = np.deg2rad([decs[i], ras[i]]),r=beam*utils.arcmin)
        stamp2 = reproject.thumbnails(map_pix,coords = np.deg2rad([decs[i], ras[i]]),r=beam*2*utils.arcmin)

        if stamp is None:
            signal.append(-9999)
            signal_ring.append(-9999)
        elif stamp[0][0][0]==0.0:
            signal.append(-9999)
            signal_ring.append(-9999)
        else:
            signal_sum = sum(sum(stamp[0]))
            signal2_sum = sum(sum(stamp2[0]))
            num_pix_signal = len(stamp[0][0])*len(stamp[0][0])
            num_pix_signal2 = len(stamp2[0][0])*len(stamp2[0][0])            
            signal.append(signal_sum/num_pix_signal)
            signal_ring.append((signal2_sum-signal_sum)/(num_pix_signal2-num_pix_signal))
        ras_used.append(ras[i])
        decs_used.append(decs[i])

    return signal, signal_ring, ras_used, decs_used


def output_signal(signal,signal_ring,output_fp,output_fp_signal):
    np.savetxt(output_fp,
               signal,
               delimiter=", ",
               fmt='% s')
    np.savetxt(output_fp_signal,
               signal_ring,
               delimiter=", ",
               fmt='% s')

In [ ]:
#real = pd.read_csv('data/union2.csv',skiprows=0,index_col=0)[::3]
ras_real = real3['amf_ra'].reset_index(drop=True)##[real.z>0.75][::2].reset_index(drop=True)
decs_real = real3['amf_dec'].reset_index(drop=True)##[real.z>0.75][::2].reset_index(drop=True)

ras_all = ras_real
decs_all = decs_real

maps = "data/HFI_SkyMap_100-field-IQU_2048_R3.00_full.fits"
beam = 10
map_pix = import_map(maps, int(beam))

outputs = "output/510u3C3/pr100.csv"
outputs2 = "output/510u3C3/pr100_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

In [ ]:
# real = pd.read_csv('data/union2.csv',skiprows=0,index_col=0)[::3]
ras_real = real3['amf_ra'].reset_index(drop=True)##[real.z>0.75][::2].reset_index(drop=True)
decs_real = real3['amf_dec'].reset_index(drop=True)##[real.z>0.75][::2].reset_index(drop=True)

ras_all = ras_real
decs_all = decs_real

maps = "data/HFI_SkyMap_143-field-IQU_2048_R3.00_full.fits"
beam = 10
map_pix = import_map(maps, int(beam))

outputs = "output/510u3C3/pr143.csv"
outputs2 = "output/510u3C3/pr143_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

In [ ]:
#real = pd.read_csv('data/union2.csv',skiprows=0,index_col=0)[::3]
ras_real = real3['amf_ra'].reset_index(drop=True)##[real.z>0.75][::2].reset_index(drop=True)
decs_real = real3['amf_dec'].reset_index(drop=True)##[real.z>0.75][::2].reset_index(drop=True)

ras_all = ras_real
decs_all = decs_real

maps = "data/HFI_SkyMap_217-field-IQU_2048_R3.00_full.fits"
beam = 10
map_pix = import_map(maps, int(beam))

outputs = "output/510u3C3/pr217.csv"
outputs2 = "output/510u3C3/pr217_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

In [ ]:
#real = pd.read_csv('data/union2.csv',skiprows=0,index_col=0)[::3]
ras_real = real3['amf_ra'].reset_index(drop=True)##[real.z>0.75][::2].reset_index(drop=True)
decs_real = real3['amf_dec'].reset_index(drop=True)##[real.z>0.75][::2].reset_index(drop=True)

ras_all = ras_real
decs_all = decs_real

maps = "data/HFI_SkyMap_353-psb-field-IQU_2048_R3.00_full.fits"
beam = 10
map_pix = import_map(maps, int(beam))

outputs = "output/510u3C3/pr353.csv"
outputs2 = "output/510u3C3/pr353_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

In [ ]:
def extract_intensity(map_pix, ras, decs, beam):
    N = len(ras)
    stack_CMB_kSZ           = 0
    stack_CMB_kSZ_deproject = 0
    c = 0

    signal = []
    signal_ring = []
    ras_used = []
    decs_used = []
    for i in tqdm(range(N)):
        dust = reproject.thumbnails(map_pix,coords = np.deg2rad([decs[i], ras[i]]),r=30*utils.arcmin)
        stamp = reproject.thumbnails(map_pix,coords = np.deg2rad([decs[i], ras[i]]),r=beam*utils.arcmin)
        stamp2 = reproject.thumbnails(map_pix,coords = np.deg2rad([decs[i], ras[i]]),r=2*beam*utils.arcmin)

        if stamp is None:
            signal.append(-9999)
            signal_ring.append(-9999)
        elif stamp[0][0][0]==0.0:
            signal.append(-9999)
            signal_ring.append(-9999)
        elif dust is None:
            signal.append(-9999)
            signal_ring.append(-9999)
        elif dust[0][0][0]==0.0:
            signal.append(-9999)
            signal_ring.append(-9999)
        else:
            dust_sum = sum(sum(dust[0]))
            num_pix_dust = len(dust[0][0])*len(dust[0][0])
            signal_sum = sum(sum(stamp[0]))
            num_pix_signal = len(stamp[0][0])*len(stamp[0][0])
            signal2_sum = sum(sum(stamp2[0]))
            num_pix_signal2 = len(stamp2[0][0])*len(stamp2[0][0])            
                        
            signal.append((signal_sum/num_pix_signal)-(dust_sum/num_pix_dust))
            signal_ring.append(((signal2_sum-signal_sum)/(num_pix_signal2-num_pix_signal))-(dust_sum/num_pix_dust))
            
        ras_used.append(ras[i])
        decs_used.append(decs[i])

    return signal, signal_ring, ras_used, decs_used

In [ ]:
#real = pd.read_csv('data/union2.csv',skiprows=0,index_col=0)[::3]
ras_real = real3['amf_ra'].reset_index(drop=True)##[real.z>0.75][::2].reset_index(drop=True)
decs_real = real3['amf_dec'].reset_index(drop=True)##[real.z>0.75][::2].reset_index(drop=True)

ras_all = ras_real
decs_all = decs_real

maps = "data/HFI_SkyMap_545-field-Int_2048_R3.00_full.fits"
beam = 10
map_pix = import_map(maps, int(beam))

outputs = "output/510u3C3/pr545.csv"
outputs2 = "output/510u3C3/pr545_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

In [ ]:
#real = pd.read_csv('data/union2.csv',skiprows=0,index_col=0)[::3]
ras_real = real3['amf_ra'].reset_index(drop=True)##[real.z>0.75][::2].reset_index(drop=True)
decs_real = real3['amf_dec'].reset_index(drop=True)##[real.z>0.75][::2].reset_index(drop=True)

ras_all = ras_real
decs_all = decs_real

maps = "data/HFI_SkyMap_857-field-Int_2048_R3.00_full.fits"
beam = 10
map_pix = import_map(maps, int(beam))

outputs = "output/510u3C3/pr857.csv"
outputs2 = "output/510u3C3/pr857_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

### Non-clusters

In [ ]:
def import_map(filepath, beam):
    pl = hp.read_map(filepath)
    pl[pl<-1e23]=0
    try:
        shape,wcs = enmap.fullsky_geometry(res=int(beam)*utils.arcmin,proj='car')
        map_pix = reproject.enmap_from_healpix(pl,shape=shape,wcs=wcs)
    except:
        shape,wcs = enmap.fullsky_geometry(res=int(beam+1)*utils.arcmin,proj='car')
        map_pix = reproject.enmap_from_healpix(pl,shape=shape,wcs=wcs)
    return map_pix


def extract_intensity(map_pix, ras, decs, beam):
    N = len(ras)
    stack_CMB_kSZ           = 0
    stack_CMB_kSZ_deproject = 0
    c = 0

    signal = []
    signal_ring = []
    ras_used = []
    decs_used = []
    for i in tqdm(range(N)):
        stamp = reproject.thumbnails(map_pix,coords = np.deg2rad([decs[i], ras[i]]),r=beam*utils.arcmin)
        stamp2 = reproject.thumbnails(map_pix,coords = np.deg2rad([decs[i], ras[i]]),r=beam*2*utils.arcmin)

        if stamp is None:
            signal.append(-9999)
            signal_ring.append(-9999)
        elif stamp[0][0][0]==0.0:
            signal.append(-9999)
            signal_ring.append(-9999)
        else:
            signal_sum = sum(sum(stamp[0]))
            signal2_sum = sum(sum(stamp2[0]))
            num_pix_signal = len(stamp[0][0])*len(stamp[0][0])
            num_pix_signal2 = len(stamp2[0][0])*len(stamp2[0][0])            
            signal.append(signal_sum/num_pix_signal)
            signal_ring.append((signal2_sum-signal_sum)/(num_pix_signal2-num_pix_signal))
        ras_used.append(ras[i])
        decs_used.append(decs[i])

    return signal, signal_ring, ras_used, decs_used


def output_signal(signal,signal_ring,output_fp,output_fp_signal):
    np.savetxt(output_fp,
               signal,
               delimiter=", ",
               fmt='% s')
    np.savetxt(output_fp_signal,
               signal_ring,
               delimiter=", ",
               fmt='% s')

In [ ]:
voids = pd.read_csv('data/voids.csv',skiprows=0,index_col=0)
ras_voids = voids['ras'].reset_index(drop=True)
decs_voids = voids['decs'].reset_index(drop=True)

ras_all = ras_voids
decs_all = decs_voids

maps = "data/HFI_SkyMap_100-field-IQU_2048_R3.00_full.fits"
beam = 10
map_pix = import_map(maps, int(beam))

outputs = "output/510u3C2/pv100.csv"
outputs2 = "output/510u3C2/pv100_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

In [ ]:
voids = pd.read_csv('data/voids.csv',skiprows=0,index_col=0)
ras_voids = voids['ras'].reset_index(drop=True)
decs_voids = voids['decs'].reset_index(drop=True)

ras_all = ras_voids
decs_all = decs_voids

maps = "data/HFI_SkyMap_143-field-IQU_2048_R3.00_full.fits"
beam = 10
map_pix = import_map(maps, int(beam))

outputs = "output/510u3C2/pv143.csv"
outputs2 = "output/510u3C2/pv143_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

In [ ]:
voids = pd.read_csv('data/voids.csv',skiprows=0,index_col=0)
ras_voids = voids['ras'].reset_index(drop=True)
decs_voids = voids['decs'].reset_index(drop=True)

ras_all = ras_voids
decs_all = decs_voids

maps = "data/HFI_SkyMap_217-field-IQU_2048_R3.00_full.fits"
beam = 10
map_pix = import_map(maps, int(beam))

outputs = "output/510u3C2/pv217.csv"
outputs2 = "output/510u3C2/pv217_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

In [ ]:
voids = pd.read_csv('data/voids.csv',skiprows=0,index_col=0)
ras_voids = voids['ras'].reset_index(drop=True)
decs_voids = voids['decs'].reset_index(drop=True)

ras_all = ras_voids
decs_all = decs_voids

maps = "data/HFI_SkyMap_353-psb-field-IQU_2048_R3.00_full.fits"
beam = 10
map_pix = import_map(maps, int(beam))

outputs = "output/510u3C2/pv353.csv"
outputs2 = "output/510u3C2/pv353_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

In [ ]:
def extract_intensity(map_pix, ras, decs, beam):
    N = len(ras)
    stack_CMB_kSZ           = 0
    stack_CMB_kSZ_deproject = 0
    c = 0

    signal = []
    signal_ring = []
    ras_used = []
    decs_used = []
    for i in tqdm(range(N)):
        dust = reproject.thumbnails(map_pix,coords = np.deg2rad([decs[i], ras[i]]),r=30*utils.arcmin)
        stamp = reproject.thumbnails(map_pix,coords = np.deg2rad([decs[i], ras[i]]),r=beam*utils.arcmin)
        stamp2 = reproject.thumbnails(map_pix,coords = np.deg2rad([decs[i], ras[i]]),r=2*beam*utils.arcmin)

        if stamp is None:
            signal.append(-9999)
            signal_ring.append(-9999)
        elif stamp[0][0][0]==0.0:
            signal.append(-9999)
            signal_ring.append(-9999)
        elif dust is None:
            signal.append(-9999)
            signal_ring.append(-9999)
        elif dust[0][0][0]==0.0:
            signal.append(-9999)
            signal_ring.append(-9999)
        else:
            dust_sum = sum(sum(dust[0]))
            num_pix_dust = len(dust[0][0])*len(dust[0][0])
            signal_sum = sum(sum(stamp[0]))
            num_pix_signal = len(stamp[0][0])*len(stamp[0][0])
            signal2_sum = sum(sum(stamp2[0]))
            num_pix_signal2 = len(stamp2[0][0])*len(stamp2[0][0])            
                        
            signal.append((signal_sum/num_pix_signal)-(dust_sum/num_pix_dust))
            signal_ring.append(((signal2_sum-signal_sum)/(num_pix_signal2-num_pix_signal))-(dust_sum/num_pix_dust))
            
        ras_used.append(ras[i])
        decs_used.append(decs[i])

    return signal, signal_ring, ras_used, decs_used

In [ ]:
voids = pd.read_csv('data/voids.csv',skiprows=0,index_col=0)
ras_voids = voids['ras'].reset_index(drop=True)
decs_voids = voids['decs'].reset_index(drop=True)

ras_all = ras_voids
decs_all = decs_voids

maps = "data/HFI_SkyMap_545-field-Int_2048_R3.00_full.fits"
beam = 10
map_pix = import_map(maps, int(beam))

outputs = "output/510u3C2/pv545.csv"
outputs2 = "output/510u3C2/pv545_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

In [ ]:
voids = pd.read_csv('data/voids.csv',skiprows=0,index_col=0)
ras_voids = voids['ras'].reset_index(drop=True)
decs_voids = voids['decs'].reset_index(drop=True)

ras_all = ras_voids
decs_all = decs_voids

maps = "data/HFI_SkyMap_857-field-Int_2048_R3.00_full.fits"
beam = 10
map_pix = import_map(maps, int(beam))

outputs = "output/510u3C2/pv857.csv"
outputs2 = "output/510u3C2/pv857_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

---

### Fullsky

In [74]:
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
from astropy import units as u

# galactic coordinates
glon = []
glat = []
for lon in np.linspace(0,180,540):
    for lat in np.linspace(15,90,224):
        glon.append(lon)
        glat.append(lat)
glon2 = [i*u.degree for i in glon]
glat2 = [i*u.degree for i in glat]

# convert to equitorial coordinates
c = SkyCoord(glon2, glat2, frame='galactic')
ras = c.icrs.ra.deg
decs = c.icrs.dec.deg

In [2]:
def import_map(filepath, beam):
    pl = hp.read_map(filepath)
    pl[pl<-1e23]=0
    try:
        shape,wcs = enmap.fullsky_geometry(res=int(beam)*utils.arcmin,proj='car')
        map_pix = reproject.enmap_from_healpix(pl,shape=shape,wcs=wcs)
    except:
        shape,wcs = enmap.fullsky_geometry(res=int(beam+1)*utils.arcmin,proj='car')
        map_pix = reproject.enmap_from_healpix(pl,shape=shape,wcs=wcs)
    return map_pix


def extract_intensity(map_pix, ras, decs, beam):
    N = len(ras)
    stack_CMB_kSZ           = 0
    stack_CMB_kSZ_deproject = 0
    c = 0

    signal = []
    signal_ring = []
    ras_used = []
    decs_used = []
    for i in tqdm(range(N)):
        stamp = reproject.thumbnails(map_pix,coords = np.deg2rad([decs[i], ras[i]]),r=beam*utils.arcmin)
        stamp2 = reproject.thumbnails(map_pix,coords = np.deg2rad([decs[i], ras[i]]),r=beam*2*utils.arcmin)

        if stamp is None:
            signal.append(-9999)
            signal_ring.append(-9999)
        elif stamp[0][0][0]==0.0:
            signal.append(-9999)
            signal_ring.append(-9999)
        else:
            signal_sum = sum(sum(stamp[0]))
            signal2_sum = sum(sum(stamp2[0]))
            num_pix_signal = len(stamp[0][0])*len(stamp[0][0])
            num_pix_signal2 = len(stamp2[0][0])*len(stamp2[0][0])            
            signal.append(signal_sum/num_pix_signal)
            signal_ring.append((signal2_sum-signal_sum)/(num_pix_signal2-num_pix_signal))
        ras_used.append(ras[i])
        decs_used.append(decs[i])

    return signal, signal_ring, ras_used, decs_used


def output_signal(signal,signal_ring,output_fp,output_fp_signal):
    np.savetxt(output_fp,
               signal,
               delimiter=", ",
               fmt='% s')
    np.savetxt(output_fp_signal,
               signal_ring,
               delimiter=", ",
               fmt='% s')

In [3]:
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
from astropy import units as u

# galactic coordinates
glon = []
glat = []
for lon in np.linspace(180,360,540):
    for lat in np.linspace(-15,-90,224):
        glon.append(lon)
        glat.append(lat)
glon2 = [i*u.degree for i in glon]
glat2 = [i*u.degree for i in glat]

# convert to equitorial coordinates
c = SkyCoord(glon2, glat2, frame='galactic')
ras = c.icrs.ra.deg
decs = c.icrs.dec.deg

In [81]:
# ras_fsky = pd.read_csv('data/full_sky_ras.csv').reset_index(drop=True)
# decs_fsky = pd.read_csv('data/full_sky_decs.csv').reset_index(drop=True)

ras_all = np.array(ras)#np.array(ras)#ras_fsky['ras'].values
decs_all = np.array(decs)#decs_fsky['decs'].values

maps = "data/HFI_SkyMap_100-field-IQU_2048_R3.00_full.fits"
beam = 10
map_pix = import_map(maps, int(beam))

outputs = "output/510u3Q2/pf100.csv"
outputs2 = "output/510u3Q2/pf100_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

Preparing SHT
T -> alm
float64 complex128
Computing pixel positions
Computing rotated positions
Projecting


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 120960/120960 [35:27<00:00, 56.86it/s]


In [82]:
# ras_fsky = pd.read_csv('data/full_sky_ras.csv').reset_index(drop=True)
# decs_fsky = pd.read_csv('data/full_sky_decs.csv').reset_index(drop=True)

ras_all = np.array(ras)#np.array(ras)#ras_fsky['ras'].values
decs_all = np.array(decs)#decs_fsky['decs'].values

maps = "data/HFI_SkyMap_143-field-IQU_2048_R3.00_full.fits"
beam = 10
map_pix = import_map(maps, int(beam))

outputs = "output/510u3Q2/pf143.csv"
outputs2 = "output/510u3Q2/pf143_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

Preparing SHT
T -> alm
float64 complex128
Computing pixel positions
Computing rotated positions
Projecting


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 120960/120960 [35:20<00:00, 57.05it/s]


In [83]:
# ras_fsky = pd.read_csv('data/full_sky_ras.csv').reset_index(drop=True)
# decs_fsky = pd.read_csv('data/full_sky_decs.csv').reset_index(drop=True)

ras_all = np.array(ras)#np.array(ras)#ras_fsky['ras'].values
decs_all = np.array(decs)#decs_fsky['decs'].values

maps = "data/HFI_SkyMap_217-field-IQU_2048_R3.00_full.fits"
beam = 10
map_pix = import_map(maps, int(beam))

outputs = "output/510u3Q2/pf217.csv"
outputs2 = "output/510u3Q2/pf217_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

Preparing SHT
T -> alm
float64 complex128
Computing pixel positions
Computing rotated positions
Projecting


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 120960/120960 [35:51<00:00, 56.22it/s]


In [84]:
# ras_fsky = pd.read_csv('data/full_sky_ras.csv').reset_index(drop=True)
# decs_fsky = pd.read_csv('data/full_sky_decs.csv').reset_index(drop=True)

ras_all = np.array(ras)#np.array(ras)#ras_fsky['ras'].values
decs_all = np.array(decs)#decs_fsky['decs'].values

maps = "data/HFI_SkyMap_353-psb-field-IQU_2048_R3.00_full.fits"
beam = 10
map_pix = import_map(maps, int(beam))

outputs = "output/510u3Q2/pf353.csv"
outputs2 = "output/510u3Q2/pf353_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

Preparing SHT
T -> alm
float64 complex128
Computing pixel positions
Computing rotated positions
Projecting


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 120960/120960 [36:29<00:00, 55.25it/s]


In [4]:
def extract_intensity(map_pix, ras, decs, beam):
    N = len(ras)
    stack_CMB_kSZ           = 0
    stack_CMB_kSZ_deproject = 0
    c = 0

    signal = []
    signal_ring = []
    ras_used = []
    decs_used = []
    for i in tqdm(range(N)):
        dust = reproject.thumbnails(map_pix,coords = np.deg2rad([decs[i], ras[i]]),r=30*utils.arcmin)
        stamp = reproject.thumbnails(map_pix,coords = np.deg2rad([decs[i], ras[i]]),r=beam*utils.arcmin)
        stamp2 = reproject.thumbnails(map_pix,coords = np.deg2rad([decs[i], ras[i]]),r=2*beam*utils.arcmin)

        if stamp is None:
            signal.append(-9999)
            signal_ring.append(-9999)
        elif stamp[0][0][0]==0.0:
            signal.append(-9999)
            signal_ring.append(-9999)
        elif dust is None:
            signal.append(-9999)
            signal_ring.append(-9999)
        elif dust[0][0][0]==0.0:
            signal.append(-9999)
            signal_ring.append(-9999)
        else:
            dust_sum = sum(sum(dust[0]))
            num_pix_dust = len(dust[0][0])*len(dust[0][0])
            signal_sum = sum(sum(stamp[0]))
            num_pix_signal = len(stamp[0][0])*len(stamp[0][0])
            signal2_sum = sum(sum(stamp2[0]))
            num_pix_signal2 = len(stamp2[0][0])*len(stamp2[0][0])            
                        
            signal.append((signal_sum/num_pix_signal)-(dust_sum/num_pix_dust))
            signal_ring.append(((signal2_sum-signal_sum)/(num_pix_signal2-num_pix_signal))-(dust_sum/num_pix_dust))
            
        ras_used.append(ras[i])
        decs_used.append(decs[i])

    return signal, signal_ring, ras_used, decs_used

In [90]:
# ras_fsky = pd.read_csv('data/full_sky_ras.csv').reset_index(drop=True)
# decs_fsky = pd.read_csv('data/full_sky_decs.csv').reset_index(drop=True)

ras_all = np.array(ras)#np.array(ras)#ras_fsky['ras'].values
decs_all = np.array(decs)#decs_fsky['decs'].values

maps = "data/HFI_SkyMap_545-field-Int_2048_R3.00_full.fits"
beam = 10
map_pix = import_map(maps, int(beam))

outputs = "output/510u3Q4/pf545.csv"
outputs2 = "output/510u3Q4/pf545_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

Preparing SHT
T -> alm
float64 complex128
Computing pixel positions
Computing rotated positions
Projecting


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 120960/120960 [54:20<00:00, 37.10it/s]


In [5]:
# ras_fsky = pd.read_csv('data/full_sky_ras.csv').reset_index(drop=True)
# decs_fsky = pd.read_csv('data/full_sky_decs.csv').reset_index(drop=True)

ras_all = np.array(ras)#np.array(ras)#ras_fsky['ras'].values
decs_all = np.array(decs)#decs_fsky['decs'].values

maps = "data/HFI_SkyMap_857-field-Int_2048_R3.00_full.fits"
beam = 10
map_pix = import_map(maps, int(beam))

outputs = "output/510u3Q4/pf857.csv"
outputs2 = "output/510u3Q4/pf857_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

Preparing SHT
T -> alm
float64 complex128
Computing pixel positions
Computing rotated positions
Projecting


100%|███████████████████████████████████████████████████████████████████████████| 120960/120960 [48:55<00:00, 41.21it/s]


In [72]:
signal_pd = pd.Series(signal)
values = signal_pd[signal_pd == -9999].index
120960-len(values)

120884

---

### ACT

### Real

In [4]:
# Import map is not used in ACT calcs below, so it is same as planck. Refer to cluster code for short updated version

def import_map(filepath, beam):
    pl = hp.read_map(filepath)
    pl[pl<-1e23]=0
    try:
        shape,wcs = enmap.fullsky_geometry(res=int(beam)*utils.arcmin,proj='car')
        map_pix = reproject.enmap_from_healpix(pl,shape=shape,wcs=wcs)
    except:
        shape,wcs = enmap.fullsky_geometry(res=int(beam+1)*utils.arcmin,proj='car')
        map_pix = reproject.enmap_from_healpix(pl,shape=shape,wcs=wcs)
    return map_pix


def extract_intensity(map_pix, ras, decs, beam):
    N = len(ras)
    stack_CMB_kSZ           = 0
    stack_CMB_kSZ_deproject = 0
    c = 0

    signal = []
    signal_ring = []
    ras_used = []
    decs_used = []
    for i in tqdm(range(N)):
        stamp = reproject.thumbnails(map_pix,coords = np.deg2rad([decs[i], ras[i]]),r=beam*utils.arcmin)
        stamp2 = reproject.thumbnails(map_pix,coords = np.deg2rad([decs[i], ras[i]]),r=beam*2*utils.arcmin)

        if stamp is None:
            signal.append(-9999)
            signal_ring.append(-9999)
        elif stamp[0][0][0]==0.0:
            signal.append(-9999)
            signal_ring.append(-9999)
        else:
            signal_sum = sum(sum(stamp[0]))
            signal2_sum = sum(sum(stamp2[0]))
            num_pix_signal = len(stamp[0][0])*len(stamp[0][0])
            num_pix_signal2 = len(stamp2[0][0])*len(stamp2[0][0])            
            signal.append(signal_sum/num_pix_signal)
            signal_ring.append((signal2_sum-signal_sum)/(num_pix_signal2-num_pix_signal))
        ras_used.append(ras[i])
        decs_used.append(decs[i])

    return signal, signal_ring, ras_used, decs_used


def output_signal(signal,signal_ring,output_fp,output_fp_signal):
    np.savetxt(output_fp,
               signal,
               delimiter=", ",
               fmt='% s')
    np.savetxt(output_fp_signal,
               signal_ring,
               delimiter=", ",
               fmt='% s')

In [ ]:
# real = pd.read_csv('data/union2.csv',skiprows=0,index_col=0)[::3]
ras_real = real3['amf_ra'].reset_index(drop=True)##[real.z>0.75][::2].reset_index(drop=True)
decs_real = real3['amf_dec'].reset_index(drop=True)##[real.z>0.75][::2].reset_index(drop=True)

ras_all = ras_real
decs_all = decs_real

beam = 5
map_pix = enmap.read_map("data/act_dr5.01_s08s18_AA_f090_daynight_map.fits")
outputs = "output/510u3C3/ar90.csv"
outputs2 = "output/510u3C3/ar90_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

In [ ]:
# real = pd.read_csv('data/union2.csv',skiprows=0,index_col=0)[::3]
ras_real = real3['amf_ra'].reset_index(drop=True)##[real.z>0.75][::2].reset_index(drop=True)
decs_real = real3['amf_dec'].reset_index(drop=True)##[real.z>0.75][::2].reset_index(drop=True)

ras_all = ras_real
decs_all = decs_real

beam = 5
map_pix = enmap.read_map("data/act_dr5.01_s08s18_AA_f150_daynight_map.fits")
outputs = "output/510u3C3/ar150.csv"
outputs2 = "output/510u3C3/ar150_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

In [ ]:
# real = pd.read_csv('data/union2.csv',skiprows=0,index_col=0)[::3]
ras_real = real3['amf_ra'].reset_index(drop=True)##[real.z>0.75][::2].reset_index(drop=True)
decs_real = real3['amf_dec'].reset_index(drop=True)##[real.z>0.75][::2].reset_index(drop=True)

ras_all = ras_real
decs_all = decs_real

beam = 5
map_pix = enmap.read_map("data/act_dr5.01_s08s18_AA_f220_daynight_map.fits")
outputs = "output/510u3C3/ar220.csv"
outputs2 = "output/510u3C3/ar220_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

### Non-clusters

In [ ]:
voids = pd.read_csv('data/voids.csv',skiprows=0,index_col=0)
ras_voids = voids['ras'].reset_index(drop=True)
decs_voids = voids['decs'].reset_index(drop=True)

ras_all = ras_voids
decs_all = decs_voids

beam = 5
map_pix = enmap.read_map("data/act_dr5.01_s08s18_AA_f090_daynight_map.fits")
outputs = "output/510u3/av90.csv"
outputs2 = "output/510u3/av90_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

In [ ]:
voids = pd.read_csv('data/voids.csv',skiprows=0,index_col=0)
ras_voids = voids['ras'].reset_index(drop=True)
decs_voids = voids['decs'].reset_index(drop=True)

ras_all = ras_voids
decs_all = decs_voids

beam = 5
map_pix = enmap.read_map("data/act_dr5.01_s08s18_AA_f150_daynight_map.fits")
outputs = "output/510u3/av150.csv"
outputs2 = "output/510u3/av150_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

In [ ]:
voids = pd.read_csv('data/voids.csv',skiprows=0,index_col=0)
ras_voids = voids['ras'].reset_index(drop=True)
decs_voids = voids['decs'].reset_index(drop=True)

ras_all = ras_voids
decs_all = decs_voids

beam = 5
map_pix = enmap.read_map("data/act_dr5.01_s08s18_AA_f220_daynight_map.fits")
outputs = "output/510u3/av220.csv"
outputs2 = "output/510u3/av220_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

---

### Fullsky

In [ ]:
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
from astropy import units as u
from tqdm import tqdm

# galactic coordinates
glon = []
glat = []
for lon in tqdm(np.linspace(180,360,1079)):
    for lat in np.linspace(15,90,449):
        if (lat >= -15) & (lat <= 15):
            continue
        glon.append(lon)
        glat.append(lat)
glon2 = [i*u.degree for i in glon]
glat2 = [i*u.degree for i in glat]

# convert to equitorial coordinates
c = SkyCoord(glon2, glat2, frame='galactic')
ras = c.icrs.ra.deg
decs = c.icrs.dec.deg

In [5]:
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
from astropy import units as u

jiggles = pd.read_csv('output/Locations/C3_wA/Allsky/jiggles_80.csv', skiprows = 0)
# jiggles[jiggles.glon>360] = 360
# jiggles[jiggles.glat>90] = 90

glon = jiggles.glon.values
glat = jiggles.glat.values

glon2 = [i*u.degree for i in glon]
glat2 = [i*u.degree for i in glat]

c = SkyCoord(glon2, glat2, frame='galactic')
ras = c.icrs.ra.deg
decs = c.icrs.dec.deg

In [ ]:
# ras_fsky = pd.read_csv('data/full_sky_ras.csv').reset_index(drop=True)
# decs_fsky = pd.read_csv('data/full_sky_decs.csv').reset_index(drop=True)

ras_all = np.array(ras)#np.array(ras)#ras_fsky['ras'].values
decs_all = np.array(decs)#decs_fsky['decs'].values

beam = 5
map_pix = enmap.read_map("data/act_dr5.01_s08s18_AA_f090_daynight_map.fits")
outputs = "output/Locations/C3_wA/Allsky/JigglesData/af90.csv"
outputs2 = "output/Locations/C3_wA/Allsky/JigglesData/af90_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

 19%|█████████████                                                         | 161502/869311 [3:20:57<17:18:07, 11.36it/s]

In [ ]:
# ras_fsky = pd.read_csv('data/full_sky_ras.csv').reset_index(drop=True)
# decs_fsky = pd.read_csv('data/full_sky_decs.csv').reset_index(drop=True)

ras_all = np.array(ras)#np.array(ras)#ras_fsky['ras'].values
decs_all = np.array(decs)#decs_fsky['decs'].values

beam = 5
map_pix = enmap.read_map("data/act_dr5.01_s08s18_AA_f150_daynight_map.fits")
outputs = "output/Locations/C3_wA/Allsky/JigglesData/af150.csv"
outputs2 = "output/Locations/C3_wA/Allsky/JigglesData/af150_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

In [ ]:
# ras_fsky = pd.read_csv('data/full_sky_ras.csv').reset_index(drop=True)
# decs_fsky = pd.read_csv('data/full_sky_decs.csv').reset_index(drop=True)

ras_all = np.array(ras)#np.array(ras)#ras_fsky['ras'].values
decs_all = np.array(decs)#decs_fsky['decs'].values

beam = 5
map_pix = enmap.read_map("data/act_dr5.01_s08s18_AA_f220_daynight_map.fits")
outputs = "output/Locations/C3_wA/Allsky/JigglesData/af220.csv"
outputs2 = "output/Locations/C3_wA/Allsky/JigglesData/af220_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

---

## Rosat

### Real

In [ ]:
def import_map(beam):
    ros = hp.read_map("data/rass_healpix_countsec.fits")
    mask = hp.read_map("data/mask_comprass.fits")

    ros[ros<-1e23]=0

    masked_ros = []
    for i in tqdm(range(len(mask))):
        masked_ros.append(0)
        if mask[i]:
            masked_ros[i]=ros[i]

    masked_ros = np.array(masked_ros)

    shape,wcs = enmap.fullsky_geometry(res=beam * utils.arcmin,proj='car')
    map_pix = reproject.enmap_from_healpix(masked_ros,shape=shape,wcs=wcs)

    return map_pix

In [ ]:
real = pd.read_csv('data/union2.csv',skiprows=0,index_col=0)[::3]
ras_real = real['amf_ra'].reset_index(drop=True)##[real.z>0.75][::2].reset_index(drop=True)
decs_real = real['amf_dec'].reset_index(drop=True)##[real.z>0.75][::2].reset_index(drop=True)

ras_all = ras_real
decs_all = decs_real

beam = 5
map_pix = import_map(int(beam))
outputs = "output/510u3/rr.csv"
outputs2 = "output/510u3/rr_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

In [ ]:
voids = pd.read_csv('data/voids.csv',skiprows=0,index_col=0)
ras_voids = voids['ras'].reset_index(drop=True)
decs_voids = voids['decs'].reset_index(drop=True)

ras_all = ras_voids
decs_all = decs_voids

beam = 5
map_pix = import_map(int(beam))
outputs = "output/510u3/rv.csv"
outputs2 = "output/510u3/rv_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)

In [ ]:
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
from astropy import units as u
from tqdm import tqdm

# galactic coordinates
glon = []
glat = []
for lon in tqdm(np.linspace(180,360,1079)):
    for lat in np.linspace(15,90,449):
        if (lat >= -15) & (lat <= 15):
            continue
        glon.append(lon)
        glat.append(lat)
glon2 = [i*u.degree for i in glon]
glat2 = [i*u.degree for i in glat]

# convert to equitorial coordinates
c = SkyCoord(glon2, glat2, frame='galactic')
ras = c.icrs.ra.deg
decs = c.icrs.dec.deg

In [ ]:
ras_fsky = pd.read_csv('data/full_sky_ras.csv').reset_index(drop=True)
decs_fsky = pd.read_csv('data/full_sky_decs.csv').reset_index(drop=True)

ras_all = np.array(ras)#np.array(ras)#ras_fsky['ras'].values
decs_all = np.array(decs)#decs_fsky['decs'].values

beam = 5
map_pix = import_map(int(beam))
outputs = "output/510u3/rf.csv"
outputs2 = "output/510u3/rf_r.csv"

signal, signal_ring, ras_used, decs_used = extract_intensity(map_pix, ras_all, decs_all, beam)
output_signal(signal, signal_ring, outputs, outputs2)